In [ ]:
OID = 609789561081430049

In [ ]:
import lsdb

cat = lsdb.open_catalog(
    "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
    filters=[("diaObjectId", "==", OID)]
)
row = cat.compute().iloc[0]
row

In [ ]:
lc = row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
)
lc

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from fit_bb import fit_mags

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

mag_mean = {
    # https://vizier.cds.unistra.fr/viz-bin/VizieR-5?-ref=VIZ685c090c15faa3&-out.add=.&-source=I/355/gaiadr3&-c=053.12047965634%20-28.32755375075,eq=ICRS,rs=2&-out.orig=o
    # Convert to AB mags, table 3 of https://www.aanda.org/articles/aa/pdf/2021/05/aa39587-20.pdf
    "G": 19.929370 + 25.8010 - 25.6874,
    "BP": 21.275953 + 25.1040 - 24.7479,
    "RP": 18.588573 + 25.3540 - 25.3385,
}
mag_std = {
    "G": 0.005559,
    "BP": 0.110131,
    "RP": 0.019574,
}

for band in "grizy":
    data = lc.query("band == @band")
    plt.errorbar(data["midpointMjdTai"], data["psfMag"], yerr=data["psfMagErr"], label=f"{band}", fmt='o', color=COLORS[band])
    median_mag = np.median(data["psfMag"])
    median_magerr = np.median(data["psfMagErr"])
    no_outliers = data.query("abs(psfMag - @median_mag) <= 5 * @median_magerr")
    mag_mean[band] = np.average(no_outliers["psfMag"], weights=no_outliers["psfMagErr"]**-2)
    mag_std[band] = np.hypot(
        1.0 / np.sqrt(np.mean(no_outliers["psfMagErr"]**-2)),
        np.std(no_outliers["psfMag"], ddof=1),
    )
    print(f"quiescence {band} mag = {mag_mean[band]:.3f} ± {mag_std[band]:.3f}")

plt.gca().invert_yaxis()
plt.xlabel("MJD")
plt.ylabel("PSF mag")
plt.title(f"diaObjectID {OID}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='lower right')
# plt.savefig(f"{OID}.pdf")

bands = "grizy"
result = fit_mags(bands, [mag_mean[b] for b in bands], [mag_std[b] for b in bands], T_init=3000)
temperature, lg_solid_angle = result.x
residual_var = np.sum(result.fun ** 2) / (len(bands) - len(result.x))
print(f"{residual_var = :.2f}")
cov = np.linalg.inv(result.jac.T @ result.jac) * residual_var
temperature_err, lg_solid_angle_err = np.sqrt(np.diag(cov))
print(f"Optimal temperature: {temperature:.2f} K ± {temperature_err:.2f} K")
print(f"Optimal log solid angle: {lg_solid_angle:.2f} log(steradian) ± {lg_solid_angle_err:.2f} log(steradian)")
print(f"Optimal solid angle: {10**lg_solid_angle:.2e} steradian")
print(f"Square root of the sum of squared residuals: {np.sqrt(result.cost):.2f}")
print("Covariance matrix:")
print(cov)

In [ ]:
print(f"r-i = {mag_mean["r"] -  mag_mean["i"]:.2f} ± {np.hypot(mag_std["r"], mag_std["i"]):.2f}")
print(f"i-z = {mag_mean["i"] -  mag_mean["z"]:.2f} ± {np.hypot(mag_std["i"], mag_std["z"]):.2f}")

In [ ]:
import astropy.units as u

# https://en.wikipedia.org/wiki/Red_dwarf
r = 0.1
d = (r * u.R_sun) / (np.tan(np.sqrt(10**lg_solid_angle * u.steradian / np.pi)))
d_pc = d.to(u.pc)
d_rel_err = np.hypot(10**lg_solid_angle_err - 1, temperature_err / temperature)
print(f"Distance: {d_pc:.2f} ± {d_pc * d_rel_err:.2f}")

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

t0 = 60646

for band in "gr":
    # data = lc.query(f"band == '{band}' and 60646.05 < midpointMjdTai < 60646.11 and psfMagErr < 0.2")
    data = lc.query(f"band == '{band}' and 60646.092 < midpointMjdTai < 60646.11 and psfMagErr < 0.2")
    plt.errorbar(data["midpointMjdTai"] - t0, data["psfMag"], yerr=data["psfMagErr"], label=f"{band}", fmt='o', color=COLORS[band])
    display(data)

plt.gca().invert_yaxis()
plt.xlabel(f"MJD $-$ {t0:d}")
plt.ylabel("PSF mag")
plt.title(f"diaObjectID {OID}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='lower right')
plt.savefig(f"{OID}.pdf")

idx_g_peak = lc.query("band == 'g' and 60646.05 < midpointMjdTai < 60646.11")["psfMag"].idxmin()
print(f"g-band peak: MJD {lc["midpointMjdTai"].loc[idx_g_peak]:.5f}, mag = {lc["psfMag"].loc[idx_g_peak]:.2f} ± {lc["psfMagErr"].loc[idx_g_peak]:.2f} amplitude = {mag_mean["g"] - lc["psfMag"].loc[idx_g_peak]:.2f} ± {np.hypot(mag_std["g"], lc["psfMagErr"].loc[idx_g_peak]):.2f}")

In [ ]:
from light_curve import RainbowFit
from light_curve.light_curve_py.features.rainbow.temperature import ConstantTemperatureTerm
from light_curve.light_curve_py.features.rainbow.bolometric import DoublexpBolometricTerm, LinexpBolometricTerm
import matplotlib.pyplot as plt
import numpy as np

COLORS = {'u': "#0e1116", 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

# effective wavelength from
# https://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?mode=browse&gname=LSST&asttype=
band_wave_aa = {"g": 4740.66, "r": 6172.34}
rainbow = RainbowFit.from_angstrom(
    band_wave_aa,
    with_baseline=True,
    temperature=ConstantTemperatureTerm(),
    # bolometric=DoublexpBolometricTerm(),
)
data = lc.query(f"60646.05 < midpointMjdTai < 60646.0975 and psfMagErr < 0.2")
params = rainbow(
    data["midpointMjdTai"].to_numpy(),
    data["psfDiffFlux"].to_numpy(),
    sigma=data["psfDiffFluxErr"].to_numpy() + 0.01 * data["psfDiffFlux"].to_numpy(),
    band=data["band"].to_numpy()
)

t_ = np.linspace(data["midpointMjdTai"].min(), data["midpointMjdTai"].max(), 1000)

for band in "gr":
    data = lc.query(f"band == '{band}' and 60646.05 < midpointMjdTai < 60646.0975 and psfMagErr < 0.2")
    plt.errorbar(data["midpointMjdTai"], data["psfDiffFlux"], yerr=data["psfDiffFluxErr"], label=f"{band} data", fmt='o', color=COLORS[band])
    plt.plot(
        t_,
        rainbow.model(t_, band, *params),
        color=COLORS[band],
        ls='--',
        label=f"{band} model"
    )

plt.xlabel("MJD")
plt.ylabel("PSF Diff Flux, nJy")
plt.title(f"diaObjectID {OID}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$")
plt.legend(loc='upper left')
# plt.savefig(f"{OID}.pdf")
print(dict(zip(rainbow.names, params.tolist())))